In [19]:
# !pip install cx_Oracle
import cx_Oracle
import random
import requests
import bs4
from bs4 import BeautifulSoup
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import euclidean_distances
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 100)
pd.set_option('display.max_columns', 100)

In [20]:
# id / pw @ host : port / service_name
connection = cx_Oracle.connect('green/1234@localhost:1521/orcl')

cursor = connection.cursor()

cursor.execute('SELECT * FROM members')

for row in cursor:
    print(row)

('python', '1234', '파이썬', datetime.datetime(2222, 2, 22, 0, 0), 'python@naver.com', '010-0232-0023', datetime.datetime(2023, 4, 11, 11, 50, 50), None, None, None, 'noImage.jpg', 0, 0, 0.0, 0, 0, 'N')
('py', '1234', '파이썬', None, 'python@naver.com', '010-3212-4564', datetime.datetime(2023, 4, 11, 13, 17, 52), None, None, None, 'noImage.jpg', 0, 0, 0.0, 0, 0, 'N')
('yjuosnpovv', '1234', '무서운 치즈', None, 'yjuosnpovv@naver.com', '010-5532-4078', datetime.datetime(2023, 4, 11, 13, 54, 19), None, None, None, 'noImage.jpg', 0, 0, 0.0, 0, 0, 'N')
('orntwmnldh', '1234', '고운 구름', None, 'orntwmnldh@naver.com', '010-8849-6337', datetime.datetime(2023, 4, 11, 13, 54, 37), None, None, None, 'noImage.jpg', 0, 0, 0.0, 0, 0, 'N')
('blicpqssmx', '1234', '짙은 기니피그', None, 'blicpqssmx@naver.com', '010-6303-4776', datetime.datetime(2023, 4, 11, 13, 54, 42), None, None, None, 'noImage.jpg', 0, 0, 0.0, 0, 0, 'N')
('mvhgixrgpb', '1234', '약빠른 망고', None, 'mvhgixrgpb@naver.com', '010-9390-4963', datetime.datetime(2

In [4]:
chars = 'abcdefghijklmnopqrstuvwxyz'
nums = '1234567890'

def random_id(length):
    result = ""
    for i in range(length):
        result += random.choice(chars)
    return result

def random_email(length):
    result = ""
    for i in range(length-2):
        result += random.choice(chars)
    result += random.choice(nums)
    result += random.choice(nums)
    result += "@naver.com"
    return result

nums = '1234567890'
def random_tel():
    result = "010-"
    for i in range(4):
        result += random.choice(nums)
    result += "-"
    for i in range(4):
        result += random.choice(nums)
    return str(result)

headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.104 Safari/537.36"}
url = f"https://ko.wiktionary.org/wiki/%EB%B6%84%EB%A5%98:%ED%95%9C%EA%B5%AD%EC%96%B4_%EA%B4%80%ED%98%95%EC%82%AC%ED%98%95(%ED%98%95%EC%9A%A9%EC%82%AC)"
res = requests.get(url, headers = headers)
res.raise_for_status()
soup = BeautifulSoup(res.text, "lxml")
proList=[]
for i in range(185):
    proList.append(soup.select('ul > li > a')[i].text)

nauList = ['개구리', '귤', '치즈', '햄스터', '구름', '망고', '강아지', '앵두', '비스킷', '우유', '기니피그']

def random_nick():
    result = ""
    result += random.choice(proList)
    result += " "
    result += random.choice(nauList)
    return result

def random_birthday():
    now = datetime.now()
    age = random.randint(18, 40)
    birthday = now - timedelta(days=age*365)
    return birthday.strftime('%Y-%m-%d')

gameList=[0, 1, 2, 3, 4]
def random_genre():
    genres=[]
    genre1 = random.choice(gameList)
    genre2 = random.choice(gameList)
    if genre1 != genre2:
        genres.append(genre1)
        genres.append(genre2)
        return genres
    else:
        return random_genre()
    
def memberGenerator(times):
    for i in range(times):
        members_id = random_id(10)
        password = "1234"
        nickname = random_nick()
        email = members_id+"@naver.com"
        tel = random_tel()
        regdate = "sysdate"
        birthday = "TO_DATE('%s', 'YYYY-MM-DD')" % random_birthday()
        genre1, genre2 = random_genre()
        
        insertQuery = "INSERT INTO members(members_id, password, nickname, email, tel, regdate, birth, genre1, genre2) VALUES('%s', '%s', '%s', '%s', '%s', %s, %s, %s, %s)" % (members_id, password, nickname, email, tel, regdate, birthday, genre1, genre2)
        print(insertQuery)
        
        cursor.execute(insertQuery)
    cursor.execute('commit')

# <hr>

In [21]:
userData_df = pd.read_csv('userData.csv').dropna()

In [22]:
X = userData_df[['Day', 'TimeSlot', 'PlayTime', 'PreferedGame', 'Gender', 'Age']]
y = userData_df['UserType']

X = pd.get_dummies(X, columns=['Day', 'TimeSlot', 'PreferedGame', 'Gender'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

knn = KNeighborsClassifier(n_neighbors=3, leaf_size=10)
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)

print(y_pred)
print('정확도:', accuracy)

[1 2 1 3 3 1 3 1 1 1 2 1 1 1 3 1 1 1 2 3 1 2 3 3 3 3 1 1 2 1 1 1 3 1 1 2 3
 3 1 1 3 1 1 3 3 1 1 3 1 1 3]
정확도: 0.803921568627451


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [23]:
weights = {'Day': 0.3, 'TimeSlot': 0.3, 'PlayTime': 0.2, 'PreferedGame': 0.5, 'Gender': 0.2, 'Age': 0.1, 'UserType': 0.5}
user_vectors = []

for index, row in userData_df.iterrows():
    vector = []
    
    for features in ['Day', 'TimeSlot', 'PlayTime', 'PreferedGame', 'Gender', 'Age', 'UserType']:
        vector.append(row[features] * weights[features])
    
    user_vectors.append(vector)

vector_distance = euclidean_distances(user_vectors)

print(vector_distance)

[[0.00000000e+00 1.91572441e+00 8.42614977e-01 ... 6.16441400e-01
  1.35277493e+00 5.96046448e-08]
 [1.91572441e+00 0.00000000e+00 1.61864141e+00 ... 1.84119526e+00
  1.33416641e+00 1.91572441e+00]
 [8.42614977e-01 1.61864141e+00 0.00000000e+00 ... 7.93725393e-01
  9.27361850e-01 8.42614977e-01]
 ...
 [6.16441400e-01 1.84119526e+00 7.93725393e-01 ... 0.00000000e+00
  1.08166538e+00 6.16441400e-01]
 [1.35277493e+00 1.33416641e+00 9.27361850e-01 ... 1.08166538e+00
  0.00000000e+00 1.35277493e+00]
 [5.96046448e-08 1.91572441e+00 8.42614977e-01 ... 6.16441400e-01
  1.35277493e+00 0.00000000e+00]]


# <hr>

In [24]:
vd = vector_distance.copy()

for i in range(vd.shape[0]):
    row = vd[i]                       
    nonzero_values = row[row != 0]     
    sorted_idx = np.argsort(nonzero_values)   
    min_idx = sorted_idx[:5]           
    min_vals = nonzero_values[min_idx] 
    min_pos = np.unique(np.where(row == min_vals[:,None])[1]) 
    print(f"Row {i}:")
    print(f"\tMin values: {min_vals}")
    print(f"\tMin positions: {min_pos[:5]}")

Row 0:
	Min values: [5.96046448e-08 5.96046448e-08 5.83095189e-01 5.83095189e-01
 6.16441400e-01]
	Min positions: [ 70  85  87 233 248]
Row 1:
	Min values: [0.53851648 0.53851648 0.64031242 0.70710678 0.72801099]
	Min positions: [ 80  99 117 133 243]
Row 2:
	Min values: [0.36055513 0.36055513 0.6164414  0.6164414  0.67082039]
	Min positions: [ 15  72 112 195 235]
Row 3:
	Min values: [0.36055513 0.36055513 0.50990195 0.50990195 0.50990195]
	Min positions: [ 40  63  73 203 226]
Row 4:
	Min values: [0.69282032 0.69282032 0.71414284 0.71414284 0.72801099]
	Min positions: [ 79 145 148 189 242]
Row 5:
	Min values: [0.36055513 0.36055513 0.36055513 0.4        0.4       ]
	Min positions: [ 32  49 122 181 212]
Row 6:
	Min values: [0.50990195 0.59160798 0.59160798 0.64031242 0.64031242]
	Min positions: [ 14  83 146 194 246]
Row 7:
	Min values: [0.41231056 0.7        0.7        0.74833148 0.74833148]
	Min positions: [ 20  40  63  73 203]
Row 8:
	Min values: [0.41231056 0.41231056 0.41231056 0.412

In [25]:
close_position={}
for i in range(vd.shape[0]):
    row = vd[i]                       
    nonzero_values = row[row != 0]     
    sorted_idx = np.argsort(nonzero_values)   
    min_idx = sorted_idx[:5]           
    min_vals = nonzero_values[min_idx] 
    min_pos = np.unique(np.where(row == min_vals[:,None])[1])
    close_position.update({i:[min_pos[:5]]})
    
cpdf = pd.DataFrame(close_position)
close_member_df = cpdf.transpose()
close_member_df.columns = ['CloseMembers']
close_member_df

,CloseMembers
0,"[70, 85, 87, 233, 248]"
1,"[80, 99, 117, 133, 243]"
2,"[15, 72, 112, 195, 235]"
3,"[40, 63, 73, 203, 226]"
4,"[79, 145, 148, 189, 242]"
5,"[32, 49, 122, 181, 212]"
6,"[14, 83, 146, 194, 246]"
7,"[20, 40, 63, 73, 203]"
8,"[48, 155, 159, 180, 211]"
9,"[48, 79, 135, 180, 211]"


In [26]:
userData_df.insert(loc=1, column='CloseMembers', value=close_member_df)
userData_df

,MembersNo,CloseMembers,Day,TimeSlot,PlayTime,PreferedGame,Gender,Age,UserType
0,1,"[70, 85, 87, 233, 248]",2,2,5,1,1,26,1
1,2,"[80, 99, 117, 133, 243]",1,3,4,4,2,22,3
2,3,"[15, 72, 112, 195, 235]",3,4,5,2,1,25,1
3,4,"[40, 63, 73, 203, 226]",1,3,2,4,2,33,3
4,5,"[79, 145, 148, 189, 242]",1,1,3,3,1,31,1
5,6,"[32, 49, 122, 181, 212]",2,2,4,1,1,17,1
6,7,"[14, 83, 146, 194, 246]",2,1,2,1,2,16,2
7,8,"[20, 40, 63, 73, 203]",1,4,5,4,1,38,3
8,9,"[48, 155, 159, 180, 211]",1,3,2,3,1,21,1
9,10,"[48, 79, 135, 180, 211]",1,3,4,2,2,27,1


# <hr>

In [28]:
cursor.execute("select members_id from members")
member_id=[]
for row in cursor:
    member_id.append(row[0])

id_df = pd.DataFrame({'member_id':member_id})
id_df

,member_id
0,admin
1,aebxkuoqsu
2,agwuxeihob
3,aioceyogwf
4,alyamrcizt
5,asndtcaqoy
6,avsblcjfok
7,axyqngfquh
8,batwunyhlf
9,bcwmbhnndy


In [30]:
userData_df.insert(loc = 1, column='Members_Id', value=id_df)
userData_df

,MembersNo,Members_Id,CloseMembers,Day,TimeSlot,PlayTime,PreferedGame,Gender,Age,UserType
0,1,admin,"[70, 85, 87, 233, 248]",2,2,5,1,1,26,1
1,2,aebxkuoqsu,"[80, 99, 117, 133, 243]",1,3,4,4,2,22,3
2,3,agwuxeihob,"[15, 72, 112, 195, 235]",3,4,5,2,1,25,1
3,4,aioceyogwf,"[40, 63, 73, 203, 226]",1,3,2,4,2,33,3
4,5,alyamrcizt,"[79, 145, 148, 189, 242]",1,1,3,3,1,31,1
5,6,asndtcaqoy,"[32, 49, 122, 181, 212]",2,2,4,1,1,17,1
6,7,avsblcjfok,"[14, 83, 146, 194, 246]",2,1,2,1,2,16,2
7,8,axyqngfquh,"[20, 40, 63, 73, 203]",1,4,5,4,1,38,3
8,9,batwunyhlf,"[48, 155, 159, 180, 211]",1,3,2,3,1,21,1
9,10,bcwmbhnndy,"[48, 79, 135, 180, 211]",1,3,4,2,2,27,1


# <hr>

In [31]:
mmdf = pd.DataFrame(columns=['MatchingMembers'])

for i in range(len(userData_df['MembersNo'])):
    memberno = userData_df['MembersNo'][i]
    mm=[]
    
    for j in range(len(userData_df['CloseMembers'][i])):
        cms = userData_df['CloseMembers'][i][j]
        
        for k in range(len(userData_df['Members_Id'])):
            memberid = userData_df['Members_Id'][k]
            eachmemberno = userData_df['MembersNo'][k]
            
            if cms+1 == eachmemberno:
                mm.append(memberid)
                
    mmdf.at[i, 'MatchingMembers'] = mm
mmdf

,MatchingMembers
0,"[hcvzwunvzf, ictwzsexdi, ifsaiimhgo, xuupmonyfo, zgccsuispw]"
1,"[hrwfsqydfr, jmygqdtepv, lxxxdgnvwi, nldzbjdeuy, ywogpapern]"
2,"[blue, hhvkvbinbu, lkwqhoafss, ugfmbaqudc, xyagpvqpnm]"
3,"[dzsgmmfpqk, glbduqchje, hjydxmjerx, vbpxkmozci, xhoepyhugg]"
4,"[hpotszmfdr, oreumzdgwb, pdciltxwxt, toeqlqhlbz, yqtugjayet]"
5,"[dbavfeplvl, fdykbbsbol, mqevvxerzp, szgxmglciq, vyezrlxkbg]"
6,"[blicpqssmx, hwkzjhxdll, orntwmnldh, ucoqcjuhve, zffxspywmd]"
7,"[brfzeyywmo, dzsgmmfpqk, glbduqchje, hjydxmjerx, vbpxkmozci]"
8,"[fcvnwgdwha, pxscgazabd, pyokrxttxg, szehlfjbrw, vuqhgqpzdz]"
9,"[fcvnwgdwha, hpotszmfdr, nqwrhduxdc, szehlfjbrw, vuqhgqpzdz]"


In [32]:
userData_df.insert(loc=2, column='CloseMembersId', value=mmdf)
userData_df

,MembersNo,Members_Id,CloseMembersId,CloseMembers,Day,TimeSlot,PlayTime,PreferedGame,Gender,Age,UserType
0,1,admin,"[hcvzwunvzf, ictwzsexdi, ifsaiimhgo, xuupmonyfo, zgccsuispw]","[70, 85, 87, 233, 248]",2,2,5,1,1,26,1
1,2,aebxkuoqsu,"[hrwfsqydfr, jmygqdtepv, lxxxdgnvwi, nldzbjdeuy, ywogpapern]","[80, 99, 117, 133, 243]",1,3,4,4,2,22,3
2,3,agwuxeihob,"[blue, hhvkvbinbu, lkwqhoafss, ugfmbaqudc, xyagpvqpnm]","[15, 72, 112, 195, 235]",3,4,5,2,1,25,1
3,4,aioceyogwf,"[dzsgmmfpqk, glbduqchje, hjydxmjerx, vbpxkmozci, xhoepyhugg]","[40, 63, 73, 203, 226]",1,3,2,4,2,33,3
4,5,alyamrcizt,"[hpotszmfdr, oreumzdgwb, pdciltxwxt, toeqlqhlbz, yqtugjayet]","[79, 145, 148, 189, 242]",1,1,3,3,1,31,1
5,6,asndtcaqoy,"[dbavfeplvl, fdykbbsbol, mqevvxerzp, szgxmglciq, vyezrlxkbg]","[32, 49, 122, 181, 212]",2,2,4,1,1,17,1
6,7,avsblcjfok,"[blicpqssmx, hwkzjhxdll, orntwmnldh, ucoqcjuhve, zffxspywmd]","[14, 83, 146, 194, 246]",2,1,2,1,2,16,2
7,8,axyqngfquh,"[brfzeyywmo, dzsgmmfpqk, glbduqchje, hjydxmjerx, vbpxkmozci]","[20, 40, 63, 73, 203]",1,4,5,4,1,38,3
8,9,batwunyhlf,"[fcvnwgdwha, pxscgazabd, pyokrxttxg, szehlfjbrw, vuqhgqpzdz]","[48, 155, 159, 180, 211]",1,3,2,3,1,21,1
9,10,bcwmbhnndy,"[fcvnwgdwha, hpotszmfdr, nqwrhduxdc, szehlfjbrw, vuqhgqpzdz]","[48, 79, 135, 180, 211]",1,3,4,2,2,27,1


# <hr>

In [ ]:
cm = userData_df['CloseMembersId']
mi = userData_df['Members_Id']
for i in range(len(mi)):
    cm_str = ','.join(cm[i])
    mii = mi[i]
    cursor.execute("INSERT INTO RECOMMENDATIONS(members_id, cm_id) VALUES('%s', '%s')" % (mii, cm_str))
# cursor.execute("COMMIT")

In [ ]:
cursor.execute("SELECT * FROM RECOMMENDATIONS")
for row in cursor:
    print(row)